<a href="https://colab.research.google.com/github/schizohub/ML_projects/blob/main/aviasales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/aviakompaniya/train_dataset.csv", index_col=0)
test_data = pd.read_csv("/kaggle/input/aviakompaniya/test_dataset.csv", index_col=0)
submission = pd.read_csv("/kaggle/input/aviakompaniya/sample_submission.csv")

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
df = train_data.copy()

In [ ]:
new_df = test_data.copy()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,  GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [ ]:
def print_value_counts(x):
    cols = []
    columns = x.columns
    for i in columns:
        cols.append(x[i].value_counts())
    return cols
def filled_data(x):
    x['Arrival Delay in Minutes']=x.fillna(np.mean(df['Arrival Delay in Minutes']))['Arrival Delay in Minutes'].astype(int)
    return x

In [ ]:
cols = print_value_counts(df)

In [ ]:
df = filled_data(df)
new_df = filled_data(new_df)

In [ ]:
print(cols[0], '\n')
print(cols[1],'\n')
print(cols[3],'\n')
print(cols[4],'\n')
print(cols[6],'\n')
print(cols[7],'\n')
print(cols[8],'\n')
print(cols[9],'\n')
print(cols[10],'\n')
print(cols[11],'\n')
print(cols[12],'\n')
print(cols[13],'\n')
print(cols[14],'\n')
print(cols[15],'\n')
print(cols[16],'\n')
print(cols[17],'\n')
print(cols[18],'\n')
print(cols[19],'\n')
print(cols[22],'\n')

In [ ]:
df[df['Cleanliness']==0]

In [ ]:
new_df

In [ ]:
# categorical_df = pd.get_dummies(df[['Gender','Customer Type','Type of Travel','Class']])
categorical_new_df = pd.get_dummies(new_df[['Gender','Customer Type','Type of Travel','Class']])

In [ ]:
for i in categorical_new_df.columns:
    #     df[i] = categorical_df[i].astype(int).copy()
    new_df[i] = categorical_new_df[i].astype(int).copy()

In [ ]:
df = df.drop(['Gender','Customer Type','Type of Travel','Class'],axis=1)

In [ ]:
new_df.drop(['Gender','Customer Type','Type of Travel','Class'],axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
new_df.info()

In [ ]:
df.describe()

In [ ]:
import plotly.express as px
fig = px.histogram(df, x='Departure Delay in Minutes')
fig.show()

In [ ]:
fig = px.histogram(new_df, x='Departure Delay in Minutes')
fig.show()

In [ ]:
fig = px.histogram(df, x='Arrival Delay in Minutes')
fig.show()

In [ ]:
fig = px.histogram(new_df, x='Arrival Delay in Minutes')
fig.show()

In [ ]:
def find_outliers_IQR(df):
    q1=df.quantile(0.25)
    q3=df.quantile(0.75)
    IQR=q3-q1
    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
    return outliers
outliers = find_outliers_IQR(df['Departure Delay in Minutes'])
print("number of outliers: "+ str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))

In [ ]:
outliers = find_outliers_IQR(new_df['Departure Delay in Minutes'])
print("number of outliers: "+ str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))

In [ ]:
outliers = find_outliers_IQR(df['Arrival Delay in Minutes'])
print("number of outliers: "+ str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))

In [ ]:
outliers = find_outliers_IQR(new_df['Arrival Delay in Minutes'])
print("number of outliers: "+ str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))

In [ ]:
outliers = find_outliers_IQR(df['Flight Distance'])
print("number of outliers: "+ str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))

In [ ]:
outliers = find_outliers_IQR(df['On-board service'])
print("number of outliers: "+ str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))

In [ ]:
outliers = find_outliers_IQR(df['Checkin service'])
print("number of outliers: "+ str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))

In [ ]:
def impute_outliers_IQR(df):
    q1=df.quantile(0.25)
    q3=df.quantile(0.75)
    IQR=q3-q1
    upper = df[~(df>(q3+1.5*IQR))].max()
    lower = df[~(df<(q1-1.5*IQR))].min()
    df = np.where(df > upper,df.mean(),np.where(df < lower,df.mean(),df))
    return df
df['Arrival Delay in Minutes'] = impute_outliers_IQR(df['Arrival Delay in Minutes'])
df['Departure Delay in Minutes'] = impute_outliers_IQR(df['Departure Delay in Minutes'])
# new_df['Arrival Delay in Minutes'] = impute_outliers_IQR(new_df['Arrival Delay in Minutes'])
# new_df['Departure Delay in Minutes'] = impute_outliers_IQR(new_df['Departure Delay in Minutes'])
df['Flight Distance'] = impute_outliers_IQR(df['Flight Distance'])
df['On-board service'] = impute_outliers_IQR(df['On-board service'])
df['Checkin service'] = impute_outliers_IQR(df['Checkin service'])
new_df['Flight Distance'] = impute_outliers_IQR(new_df['Flight Distance'])
new_df['On-board service'] = impute_outliers_IQR(new_df['On-board service'])
new_df['Checkin service'] = impute_outliers_IQR(new_df['Checkin service'])
# df.describe()[['Arrival Delay in Minutes','Departure Delay in Minutes','Flight Distance','On-board service','Checkin service']]

In [ ]:
df

In [ ]:
new_df.describe()[['Arrival Delay in Minutes','Departure Delay in Minutes','Flight Distance','On-board service','Checkin service']]

In [ ]:
X_train, X_test = train_test_split(df, test_size=0.25, random_state=100)

In [ ]:
X_t = X_test.drop('satisfaction', axis=1)
y_t = X_test['satisfaction'].values

In [ ]:
X = X_train.drop('satisfaction',axis=1)
y = X_train['satisfaction'].values

In [ ]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X = std.fit_transform(X)
X_t = std.fit_transform(X_t)

In [ ]:
X

In [ ]:
new_X = std.fit_transform(new_df.copy())

In [ ]:
new_X

In [ ]:
X

In [ ]:
lg = LogisticRegression()
lg.fit(X,y)

In [ ]:
prediction = lg.predict(X_t)
print('Accuracy of Logistic reg: {:.2f}'.format(metrics.accuracy_score(y_t,prediction)),
'\n Jaccard score of Logistic reg: {:.2f}'.format(metrics.jaccard_score(y_t,prediction)))
print(metrics.classification_report(y_t,prediction))

conf_mat = metrics.confusion_matrix(y_t,prediction)
sns.heatmap(conf_mat, annot=True,fmt="g")
plt.show()

fpr, tpr, thresholds = metrics.roc_curve(y_t,prediction)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='ROC curve')
display.plot()
plt.show()

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X,y)

In [ ]:
prediction = knn.predict(X_t)
print('Accuracy of K-NN: {:.2f}'.format(metrics.accuracy_score(y_t,prediction)),
     '\n Jaccard score of K-NN: {:.2f}'.format(metrics.jaccard_score(y_t,prediction)))
print(metrics.classification_report(y_t,prediction))

conf_mat = metrics.confusion_matrix(y_t,prediction)
sns.heatmap(conf_mat, annot=True,fmt="g")
plt.show()

fpr, tpr, thresholds = metrics.roc_curve(y_t,prediction)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='ROC curve')
display.plot()
plt.show()

In [ ]:
rforest = RandomForestClassifier(n_estimators=110,max_depth=25)
rforest.fit(X,y)

In [ ]:
prediction = rforest.predict(X_t)
print('Accuracy of Random forest: {:.2f}'.format(metrics.accuracy_score(y_t,prediction)),
'\n Jaccard score of Random forest: {:.2f}'.format(metrics.jaccard_score(y_t,prediction)))
print(metrics.classification_report(y_t,prediction))

conf_mat = metrics.confusion_matrix(y_t,prediction)
sns.heatmap(conf_mat, annot=True,fmt="g")
plt.show()

fpr, tpr, thresholds = metrics.roc_curve(y_t,prediction)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='ROC curve')
display.plot()
plt.show()

In [ ]:
param_grid = {'n_estimators': np.arange(1, 125)}

forest_gscv = GridSearchCV(rforest, param_grid, cv=5)

forest_gscv.fit(X,y)
forest_gscv.best_params_

In [ ]:
tree = DecisionTreeClassifier(min_samples_leaf=11)
tree.fit(X,y)

In [ ]:
prediction = tree.predict(X_t)
print('Accuracy of Decision tree: {:.2f}'.format(metrics.accuracy_score(y_t,prediction)),
'\n Jaccard score of Decision tree: {:.2f}'.format(metrics.jaccard_score(y_t,prediction)))
print(metrics.classification_report(y_t,prediction))

conf_mat = metrics.confusion_matrix(y_t,prediction)
sns.heatmap(conf_mat, annot=True,fmt="g")
plt.show()

fpr, tpr, thresholds = metrics.roc_curve(y_t,prediction)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='ROC curve')
display.plot()
plt.show()

In [ ]:
xgb = XGBClassifier()
xgb.fit(X,y)

In [ ]:
prediction = xgb.predict(X_t)
print('Accuracy of Decision tree: {:.2f}'.format(metrics.accuracy_score(y_t,prediction)),
'\n Jaccard score of Decision tree: {:.2f}'.format(metrics.jaccard_score(y_t,prediction)))
print(metrics.classification_report(y_t,prediction))

conf_mat = metrics.confusion_matrix(y_t,prediction)
sns.heatmap(conf_mat, annot=True,fmt="g")
plt.show()

fpr, tpr, thresholds = metrics.roc_curve(y_t,prediction)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='ROC curve')
display.plot()
plt.show()

In [ ]:
svc = SVC()
svc.fit(X,y)

In [ ]:
prediction = svc.predict(X_t)
print('Accuracy of Decision tree: {:.2f}'.format(metrics.accuracy_score(y_t,prediction)),
'\n Jaccard score of Decision tree: {:.2f}'.format(metrics.jaccard_score(y_t,prediction)))
print(metrics.classification_report(y_t,prediction))

conf_mat = metrics.confusion_matrix(y_t,prediction)
sns.heatmap(conf_mat, annot=True,fmt="g")
plt.show()

fpr, tpr, thresholds = metrics.roc_curve(y_t,prediction)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='ROC curve')
display.plot()
plt.show()

In [ ]:
predictions = rforest.predict(new_X)

In [ ]:
prediction_xgb = xgb.predict(new_X)

In [ ]:
submission['satisfaction'] = predictions
submission.to_csv('submission_rforest.csv', index=False)

In [ ]:
submission['satisfaction'] = prediction_xgb
submission.to_csv('submission_xgb.csv', index=False)